```
在数据运算时，两个数据进行运算，那么它们必须同时存放在同一个设备，要么同时是 CPU，要么同时是 GPU。而且数据和模型都要在同一个设备上。数据和模型可以使用to()方法从一个设备转移到另一个设备。而数据的to()方法还可以转换数据类型。
```

### 从 CPU 到 GPU
```python
    device = torch.device("cuda")
    tensor = tensor.to(device)
    module.to(device)

### 从 GPU 到 CPU
```python
    device = torch.device(cpu)
    tensor = tensor.to("cpu")
    module.to("cpu")

```
tensor和module的 to()方法的区别是：tensor.to()执行的不是 inplace 操作，因此需要赋值；module.to()执行的是 inplace 操作。

#### 下面的代码是转换数据类型

In [9]:
import torch

x = torch.ones((3,3))
print(x.dtype)
x = x.to(torch.float64)
print(x.dtype)


torch.float32
torch.float64


### tensor.to() 和 module.to()

In [10]:
# 首先导入库，获取 GPU 的 device
import torch
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [11]:
# 下面的代码是执行Tensor的to()方法
x_cpu = torch.ones((3, 3))

print("x_cpu:\ndevice: {} is_cuda: {} id: {}".format(x_cpu.device, x_cpu.is_cuda, id(x_cpu)))

x_gpu = x_cpu.to(device)
print("x_gpu:\ndevice: {} is_cuda: {} id: {}".format(x_gpu.device, x_gpu.is_cuda, id(x_gpu)))

x_cpu:
device: cpu is_cuda: False id: 4456792208
x_gpu:
device: cpu is_cuda: False id: 4456792208


### torch.cuda常用方法

```
torch.cuda.device_count()：返回当前可见可用的 GPU 数量
torch.cuda.get_device_name()：获取 GPU 名称
torch.cuda.manual_seed()：为当前 GPU 设置随机种子
torch.cuda.manual_seed_all()：为所有可见 GPU 设置随机种子
torch.cuda.set_device()：设置主 GPU 为哪一个物理 GPU，此方法不推荐使用
os.environ.setdefault("CUDA_VISIBLE_DEVICES", "2", "3")：设置可见 GPU


### 在 PyTorch 中，有物理 GPU 可以逻辑 GPU 之分，可以设置它们之间的对应关系。
![image](https://image.zhangxiann.com/20200707194809.png "image")

```
在上图中，如果执行了os.environ.setdefault("CUDA_VISIBLE_DEVICES", "2", "3")，那么可见 GPU 数量只有 2 个。对应关系如下：
逻辑 GPU    gpu0    gpu1
物理 GPU    gpu2    gpu3

```
如果执行了os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0", "3", "2")，那么可见 GPU 数量只有 3 个。对应关系如下：
逻辑 GPU    gpu0    gpu1    gpu2
物理 GPU    gpu0    gpu3    gpu2

### 多 GPU 的分发并行
```python
torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)
```
```
功能：包装模型，实现分发并行机制。可以把数据平均分发到各个 GPU 上，每个 GPU 实际的数据量为 $\frac{batch_size}{GPU 数量}$，实现并行计算。
```
```
主要参数：
    module：需要包装分发的模型
    device_ids：可分发的 GPU，默认分发到所有可见可用的 GPU
    output_device：结果输出设备

#### 需要注意的是：使用 DataParallel 时，device 要指定某个 GPU 为 主 GPU，否则会报错：
RuntimeError: module must have its parameters and buffers on device cuda:1 (device_ids[0]) but found one of them on device: cuda:2

这是因为，使用多 GPU 需要有一个主 GPU，来把每个 batch 的数据分发到每个 GPU，并从每个 GPU 收集计算好的结果。如果不指定主 GPU，那么数据就直接分发到每个 GPU，会造成有些数据在某个 GPU，而另一部分数据在其他 GPU，计算出错。

### 下面的代码设置两个可见 GPU，batch_size 为 2，那么每个 GPU 每个 batch 拿到的数据数量为 8，在模型的前向传播中打印数据的数量。
```python
    # 设置 2 个可见 GPU
    gpu_list = [0,1]
    gpu_list_str = ','.join(map(str, gpu_list))
    os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
    # 这里注意，需要指定一个 GPU 作为主 GPU。
    # 否则会报错：module must have its parameters and buffers on device cuda:1 (device_ids[0]) but found one of them on device: cuda:2
    # 参考：https://stackoverflow.com/questions/59249563/runtimeerror-module-must-have-its-parameters-and-buffers-on-device-cuda1-devi
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    batch_size = 16

    # data
    inputs = torch.randn(batch_size, 3)
    labels = torch.randn(batch_size, 3)

    inputs, labels = inputs.to(device), labels.to(device)

    # model
    net = FooNet(neural_num=3, layers=3)
    net = nn.DataParallel(net)
    net.to(device)

    # training
    for epoch in range(1):

        outputs = net(inputs)

        print("model outputs.size: {}".format(outputs.size()))

    print("CUDA_VISIBLE_DEVICES :{}".format(os.environ["CUDA_VISIBLE_DEVICES"]))
    print("device_count :{}".format(torch.cuda.device_count()))

### 下面的代码是根据 GPU 剩余内存来排序。
```python
    def get_gpu_memory():
        import platform
        if 'Windows' != platform.system():
            import os
            os.system('nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt')
            memory_gpu = [int(x.split()[2]) for x in open('tmp.txt', 'r').readlines()]
            os.system('rm tmp.txt')
        else:
            memory_gpu = False
            print("显存计算功能暂不支持windows操作系统")
        return memory_gpu


    gpu_memory = get_gpu_memory()
    if not gpu_memory:
        print("\ngpu free memory: {}".format(gpu_memory))
        gpu_list = np.argsort(gpu_memory)[::-1]

        gpu_list_str = ','.join(map(str, gpu_list))
        os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

```
其中nvidia-smi -q -d Memory是查询所有 GPU 的内存信息，-q表示查询，-d是指定查询的内容。
nvidia-smi -q -d Memory | grep -A4 GPU是截取 GPU 开始的 4 行，如下

nvidia-smi -q -d Memory | grep -A4 GPU | grep Free是提取Free所在的行，也就是提取剩余内存的信息，如下：

nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt是把剩余内存的信息保存到tmp.txt中。

[int(x.split()[2]) for x in open('tmp.txt', 'r').readlines()]是用列表表达式对每行进行处理。

#### GPU详细内容链接

https://pytorch.zhangxiann.com/7-mo-xing-qi-ta-cao-zuo/7.3-shi-yong-gpu-xun-lian-mo-xing#duo-gpu-de-fen-fa-bing-hang